In [1]:
import numpy as np
import pandas as pd

In [2]:
df1 = pd.read_csv("D:/ml/deepwalk-master/kospi200/data/kospi200.csv",
                 index_col = 0,
                 header=0,
                 encoding='cp949', sep=',')

In [3]:
df2 = pd.read_csv("D:/ml/deepwalk-master/kospi200/data/samsung.csv",
                 index_col = 0,
                 header=0,
                 encoding='cp949', sep=',')

In [4]:
for i in range(len(df1.index)):
    
    df1.iloc[i,4] = int(df1.iloc[i,4].replace(',',''))

In [5]:
for i in range(len(df2.index)):
    
    for j in range(len(df2.iloc[i])):
        
        df2.iloc[i,j] = int(df2.iloc[i,j].replace(',',''))

In [6]:
df1 = df1.sort_values(['일자'], ascending=True)
df2 = df2.sort_values(['일자'], ascending=True)

In [7]:
df1['거래량'] = df1['거래량'].astype('float64')

In [8]:
df2['시가'] = df2['시가'].astype('float64')
df2['고가'] = df2['고가'].astype('float64')
df2['저가'] = df2['저가'].astype('float64')
df2['종가'] = df2['종가'].astype('float64')
df2['거래량'] = df2['거래량'].astype('float64')

In [9]:
df1 = df1.values
df2 = df2.values

In [10]:
type(df1)

numpy.ndarray

In [11]:
type(df2)

numpy.ndarray

In [12]:
kospi200 = df1.copy()
samsung = df2.copy()

In [13]:
def split_xy5(dataset, time_steps, y_column):
    x, y = list(), list()
    for i in range(len(dataset)):
        x_end_number = i + time_steps
        y_end_number = x_end_number + y_column # 수정

        if y_end_number > len(dataset):  # 수정
            break
        tmp_x = dataset[i:x_end_number, :]  # 수정
        tmp_y = dataset[x_end_number:y_end_number, 3]    # 수정
        x.append(tmp_x)
        y.append(tmp_y)
    return np.array(x), np.array(y)

In [20]:
x1, y1= split_xy5(samsung, 5,1)
x2, y2 = split_xy5(kospi200, 5,1)

In [21]:
print(x2[0,:], "\n", y2[0])

[[   320.56    321.      316.75    316.75 173911.  ]
 [   317.79    319.53    314.95    315.87 141252.  ]
 [   315.93    316.08    312.6     314.42 123610.  ]
 [   316.51    317.34    315.11    317.31 120900.  ]
 [   318.03    319.7     317.86    318.51 112400.  ]] 
 [317.72]


In [22]:
print(x2.shape)
print(y2.shape)

(421, 5, 5)
(421, 1)


In [23]:
from sklearn.model_selection import train_test_split

In [24]:
x1_train, x1_test, y1_train, y1_test = train_test_split(x1,
                                                        y1,
                                                        random_state=2020,
                                                        test_size = 0.3)
x2_train, x2_test, y2_train, y2_test = train_test_split(x2,
                                                        y2,
                                                        random_state=2020,
                                                        test_size = 0.3)

In [25]:
print(x2_train.shape)
print(x2_test.shape)
print(y2_train.shape)
print(y2_test.shape)

(294, 5, 5)
(127, 5, 5)
(294, 1)
(127, 1)


In [26]:
x1_train = np.reshape(x1_train,
                      (x1_train.shape[0], x1_train.shape[1] * x1_train.shape[2]))
x1_test = np.reshape(x1_test,
                     (x1_test.shape[0], x1_test.shape[1] * x1_test.shape[2]))
x2_train = np.reshape(x2_train,
                      (x2_train.shape[0], x2_train.shape[1] * x2_train.shape[2]))
x2_test = np.reshape(x2_test,
                     (x2_test.shape[0], x2_test.shape[1] * x2_test.shape[2]))

In [27]:
print(x2_train.shape)
print(x2_test.shape)

(294, 25)
(127, 25)


In [28]:
from sklearn.preprocessing import StandardScaler

scaler1 = StandardScaler()
scaler1.fit(x1_train)
x1_train_scaled = scaler1.transform(x1_train)
x1_test_scaled = scaler1.transform(x1_test)

scaler2 = StandardScaler()
scaler2.fit(x2_train)
x2_train_scaled = scaler2.transform(x2_train)
x2_test_scaled = scaler2.transform(x2_test)

In [29]:
print(x1_train_scaled[0, :])
print(x2_train_scaled[0, :])

[ 0.04562609  0.05104716  0.09054592  0.0020591  -0.08867535  0.10079632
  0.03721625  0.10892135  0.07459096 -0.34172486  0.10092696  0.13519501
  0.13600635  0.19633445 -0.19625155  0.11080014  0.03585534  0.00366393
 -0.02218311 -0.12886738 -0.06295995 -0.10554794 -0.04822327 -0.10779072
 -0.63323557]
[ 1.20004525  1.17942984  1.10373998  1.02696889 -0.2621014   1.09348341
  1.0228551   1.06387508  0.99297912  0.05514018  0.98583206  1.01754208
  0.98689321  1.0774234  -0.36544753  1.06090613  0.97680926  0.9800884
  0.92828959  0.16269962  0.94648243  0.92755017  0.94839905  0.99207253
  0.68705535]


In [30]:
from keras.models import Model
from keras.layers import Dense, Input

Using TensorFlow backend.


In [31]:
# 모델구성
input1 = Input(shape=(25, ))
dense1 = Dense(64)(input1)
dense1 = Dense(32)(dense1)
dense1 = Dense(32)(dense1)
output1 = Dense(32)(dense1)

In [32]:
input2 = Input(shape=(25, ))
dense2 = Dense(64)(input2)
dense2 = Dense(64)(dense2)
dense2 = Dense(64)(dense2)
dense2 = Dense(64)(dense2)
output2 = Dense(32)(dense2)

In [33]:
from keras.layers.merge import concatenate

In [34]:
merge = concatenate([output1, output2])

In [35]:
output3 = Dense(1)(merge)

In [37]:
model = Model(inputs= [input1, input2],
             outputs = output3)

In [39]:
model.compile(loss='mse', optimizer='adam', metrics=['mse'])

In [40]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 25)]         0                                            
__________________________________________________________________________________________________
input_1 (InputLayer)            [(None, 25)]         0                                            
__________________________________________________________________________________________________
dense_4 (Dense)                 (None, 64)           1664        input_2[0][0]                    
__________________________________________________________________________________________________
dense (Dense)                   (None, 64)           1664        input_1[0][0]                    
______________________________________________________________________________________________

In [41]:
#from keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import EarlyStopping
early_stopping = EarlyStopping(patience=20)

In [42]:
model.fit([x1_train_scaled, x2_train_scaled], y1_train, 
          validation_split=0.2,
         verbose=1, batch_size = 1, epochs=100,
         callbacks = [early_stopping])

Epoch 1/100
235/235 [==============================] - 0s 2ms/step - loss: 1477729664.0000 - mse: 1477729664.0000 - val_loss: 25302302.0000 - val_mse: 25302302.0000
Epoch 2/100
235/235 [==============================] - 0s 1ms/step - loss: 10311252.0000 - mse: 10311252.0000 - val_loss: 4212523.0000 - val_mse: 4212523.0000
Epoch 3/100
235/235 [==============================] - 0s 1ms/step - loss: 6327531.5000 - mse: 6327531.5000 - val_loss: 7013787.5000 - val_mse: 7013787.5000
Epoch 4/100
235/235 [==============================] - 0s 1ms/step - loss: 3547941.0000 - mse: 3547941.0000 - val_loss: 3052956.2500 - val_mse: 3052956.2500
Epoch 5/100
235/235 [==============================] - 0s 1ms/step - loss: 1906962.5000 - mse: 1906962.5000 - val_loss: 1784797.7500 - val_mse: 1784797.7500
Epoch 6/100
235/235 [==============================] - 0s 1ms/step - loss: 2227589.5000 - mse: 2227589.5000 - val_loss: 1778598.0000 - val_mse: 1778598.0000
Epoch 7/100
235/235 [===========================

In [43]:
loss, mse = model.evaluate([x1_test_scaled, x2_test_scaled], 
                           y1_test, batch_size=1)

127/127 [==============================] - 0s 735us/step - loss: 1337776.6250 - mse: 1337776.6250


In [44]:
print('loss : ', loss)
print('mse : ', mse)

loss :  1337776.625
mse :  1337776.625


In [45]:
y1_pred = model.predict([x1_test_scaled, x2_test_scaled])

for i in range(5):
    print('종가 : ', y1_test[i], '/ 예측가 : ', y1_pred[i])

종가 :  [45200.] / 예측가 :  [46151.85]
종가 :  [48400.] / 예측가 :  [49456.414]
종가 :  [46950.] / 예측가 :  [47192.316]
종가 :  [47500.] / 예측가 :  [46596.04]
종가 :  [51100.] / 예측가 :  [51811.395]
